<a href="https://colab.research.google.com/github/seungmei/ESAA_24_1/blob/main/AI_vs_Human_%ED%85%8D%EC%8A%A4%ED%8A%B8_%ED%8C%90%EB%B3%84%EC%9D%98_%EA%B2%BD%EA%B3%84_%EC%B0%BE%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI vs Human 텍스트 판별의 경계 찾기
https://dacon.io/competitions/official/236178/overview/description

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import string
import re

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/text_classification/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/text_classification/test.csv')
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/text_classification/sample_submission.csv')

In [4]:
train.head(5)

,id,sentence1,sentence2,sentence3,sentence4,label
0,TRAIN_000,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것...,직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서...,직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 ...,2
1,TRAIN_001,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,분위기가 너무 좋아요! 2층 창문이 넓어서 쾌적한 느낌이에요. 조명도 아름답고 음료...,분위기가 짱!! 2층 창문이 커서 탁 트여있는 느낌이에요 ㅎㅎ 조명도 예쁘고 음료랑...,분위기가 너무 좋아요! 2층 창문이 크고 넓어서 탁 트여있는 느낌이에요. 조명도 예...,3
2,TRAIN_002,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",일단 장사가 잘되길 바라는 마음에서 별5개 드립니다 간도 맞았고 매운걸 좋아하는 입...,일단 저는 장사가 잘되기를 바라는 마음에서 별 다섯 개를 주고 싶어요. 맛도 딱 맞...,"먼저, 칭찬과 응원의 의미로 별 다섯 개를 주고 싶습니다. 간도 딱 맞고, 저는 매...",2
3,TRAIN_003,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...","1편의 신선함에 비해 약간 빛이 바래 보이지만, 여전히 재미있게 즐길 수 있어요. ...","1편의 독특함 때문에 약간의 비교가 불가피하지만, 이 게임은 여전히 흥미로워요. 시...",1편이 워낙 참신했던 탓에 좀 묻힌 감이 있긴 하지만 재미는 여전합니다. 시스템도 ...,4
4,TRAIN_004,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",빵점을 주고 싶지만 참아냈습니다... 이 영화는 사상 유래가 없는 것 같아요. 조지...,빵점 주고 싶을 정도로 엄청 실망했어요... 이 영화는 별들의 전쟁처럼 역사적인 작...,"빵점을 주고 싶었는데 참았어요... 이런 영화는 전례가 없는데, 조지 루카스의 스타...",1


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         50 non-null     object
 1   sentence1  50 non-null     object
 2   sentence2  50 non-null     object
 3   sentence3  50 non-null     object
 4   sentence4  50 non-null     object
 5   label      50 non-null     int64 
dtypes: int64(1), object(5)
memory usage: 2.5+ KB


In [6]:
test.head(3)

,id,sentence1,sentence2,sentence3,sentence4
0,TEST_0000,"배송은 정말 빨랐어요! 집에서 속옷이 불편한데 입기 싫고, 안 입으면 민망해서 구매...","배송은 정말 빠르고요~ 집에서 속옷은 불편해서 안 입으려고 했는데, 약간 부끄럽기도...",배송은 빠르구여~집에서 속옷 불편한데 입긴싫고~안입자니 민망해서 구매했는데~오 사이...,배송 빨라요~ 속옷 불편한데 입기 싫고~ 안 입으면 민망해서 삿는데~ 사이즈 크네요...
1,TEST_0001,하.. 원래도 비쌋는데 가격도 더ㅠ올라가고 품질은 더 떨어졌어요.. 김밥에 참...,가격 상승에 비해서 품질이 떨어진 점이 눈에 띄었어요. 그 전엔 참기름과 깨로 김밥...,원래 가격이 높았음에도 불구하고 가격 상승과 품질 하락이 동시에 이루어진 것 같아요...,가격 상승과 함께 품질이 떨어진 점이 아쉬웠습니다. 예전에는 참기름과 깨를 더해 풍...
2,TEST_0002,"평일 저녁에 방문했을 때, 서비스가 좋지 않았습니다. 음료를 준비하는 듯 보여 기다...",평일 저녁에 갔는데 매우 불친절했어요. 음료하시는 것 같아서 일단 기다리는데 보고선...,평일 저녁에 방문했을 때 서비스가 매우 불만족스러웠습니다. 음료를 준비하는 듯 보였...,"평일 저녁에 찾아간 카페, 확실히 서비스는 별로였어요. 직원들은 바쁜 척 음료를 만..."


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1100 non-null   object
 1   sentence1  1100 non-null   object
 2   sentence2  1100 non-null   object
 3   sentence3  1100 non-null   object
 4   sentence4  1100 non-null   object
dtypes: object(5)
memory usage: 43.1+ KB


In [8]:
submission.head(3)

,id,label
0,TEST_0000,55
1,TEST_0001,55
2,TEST_0002,55


In [9]:
train.shape, test.shape

((50, 6), (1100, 5))

## 데이터 전처리

### 텍스트 클렌징

In [10]:
test[test['id']=='TEST_0796']

,id,sentence1,sentence2,sentence3,sentence4
796,TEST_0796,お水がエアコンの真下に放置されていてとても汚かったです。そして、席に座り注文もしてしまった後...,お水がエアコンの真下に置かれていて見た目が本当に汚かったです。それに、座って注文をした後に店...,お水がエアコンの真下に放置されていてとても不潔でした。それに、座った後に注文をしたら、「コロ...,お水がエアコンの真下に置かれていて、非常に汚かったです。そして、座って注文をしてから「コロナ...


In [11]:
# 외국어 열을 0으로 채우기
test.loc[test['id'] == 'TEST_0796', ['sentence1', 'sentence2', 'sentence3', 'sentence4']] = '0'

# 0이 채워졌는지 확인
test[test['id']=='TEST_0796']

,id,sentence1,sentence2,sentence3,sentence4
796,TEST_0796,0,0,0,0


In [12]:
test[test['id']=='TEST_0607']

,id,sentence1,sentence2,sentence3,sentence4
607,TEST_0607,"자주 먹는 음식 중에서는 마라탕을 세 번 매운맛으로 시켜먹고, 쯔마장을 추가로 부탁...",经历过一再辣三级的尝试之后，加上特别要求加上了蘑菇，和麻辣酱混合在一起尝起来真是美味无比！！...,자주 먹는 마라탕을 3단계 매운맛으로 주문하고 쯔마장을 추가해서 먹으면 정말 맛있어...,经常麻辣 3단계로 먹고 쯔마장 달라고 해서 마유랑 같이먹으면 존맛탱임 쯔마장 진짜 ...


In [13]:
test.loc[test['id'] == 'TEST_0607', 'sentence2'] = '0'

test[test['id']=='TEST_0607']

,id,sentence1,sentence2,sentence3,sentence4
607,TEST_0607,"자주 먹는 음식 중에서는 마라탕을 세 번 매운맛으로 시켜먹고, 쯔마장을 추가로 부탁...",0,자주 먹는 마라탕을 3단계 매운맛으로 주문하고 쯔마장을 추가해서 먹으면 정말 맛있어...,经常麻辣 3단계로 먹고 쯔마장 달라고 해서 마유랑 같이먹으면 존맛탱임 쯔마장 진짜 ...


In [14]:
train = train.drop(columns=["id"])

def update_label(row):
    label_column = row['label']
    if int(row['column']) == int(label_column):
        return 1
    else:
        return 0

train.columns = ["1", "2", "3", "4", "label"]
train = train.melt(id_vars='label', value_vars=train.columns[:-1], var_name='column', value_name='text')

train['label'] = train.apply(update_label, axis=1)
train.drop(columns=["column"], inplace=True)

train = pd.concat([train.iloc[:, 1], train.iloc[:, 0]], axis=1)

In [15]:
train.head()

,text,label
0,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",0
1,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,0
2,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",0
3,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...",0
4,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",1


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    200 non-null    object
 1   label   200 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 3.2+ KB


In [17]:
test = test.drop(columns=["id"])

test = test.reset_index()
test = pd.melt(test, id_vars='index', var_name="sentence", value_name="text")

In [18]:
test.head()

,index,sentence,text
0,0,sentence1,"배송은 정말 빨랐어요! 집에서 속옷이 불편한데 입기 싫고, 안 입으면 민망해서 구매..."
1,1,sentence1,하.. 원래도 비쌋는데 가격도 더ㅠ올라가고 품질은 더 떨어졌어요.. 김밥에 참...
2,2,sentence1,"평일 저녁에 방문했을 때, 서비스가 좋지 않았습니다. 음료를 준비하는 듯 보여 기다..."
3,3,sentence1,"직원들의 친절도가 좋지 않았고, 손님이 오셨음에도 불구하고 무시하는 태도를 보여서 ..."
4,4,sentence1,갈비를 주문했음에도 다른 메뉴가 나왔어요. 그런데 고기의 형태가 기대와 달라 실망스...


In [19]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4400 entries, 0 to 4399
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     4400 non-null   int64 
 1   sentence  4400 non-null   object
 2   text      4400 non-null   object
dtypes: int64(1), object(2)
memory usage: 103.2+ KB


In [20]:
'''
# 텍스트 클렌징 함수 정의
def clean_text(text):
    # 특수문자 제거
    text = re.sub(r'[^가-힣A-Za-z0-9 ]', '', text)

    # 여러 개의 공백은 하나로 줄이기
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# 클렌징 함수를 train과 test에 적용
train['sentence1'] = train['sentence1'].apply(clean_text)
train['sentence2'] = train['sentence2'].apply(clean_text)
train['sentence3'] = train['sentence3'].apply(clean_text)
train['sentence4'] = train['sentence4'].apply(clean_text)

test['sentence1'] = test['sentence1'].apply(clean_text)
test['sentence2'] = test['sentence2'].apply(clean_text)
test['sentence3'] = test['sentence3'].apply(clean_text)
test['sentence4'] = test['sentence4'].apply(clean_text)
'''

"\n# 텍스트 클렌징 함수 정의\ndef clean_text(text):\n    # 특수문자 제거\n    text = re.sub(r'[^가-힣A-Za-z0-9 ]', '', text)\n\n    # 여러 개의 공백은 하나로 줄이기\n    text = re.sub(r'\\s+', ' ', text).strip()\n\n    return text\n\n# 클렌징 함수를 train과 test에 적용\ntrain['sentence1'] = train['sentence1'].apply(clean_text)\ntrain['sentence2'] = train['sentence2'].apply(clean_text)\ntrain['sentence3'] = train['sentence3'].apply(clean_text)\ntrain['sentence4'] = train['sentence4'].apply(clean_text)\n\ntest['sentence1'] = test['sentence1'].apply(clean_text)\ntest['sentence2'] = test['sentence2'].apply(clean_text)\ntest['sentence3'] = test['sentence3'].apply(clean_text)\ntest['sentence4'] = test['sentence4'].apply(clean_text)\n"

In [21]:
'''
# train dataframe을 분석하기 좋게 변환하기
# 레이블에 맞는 sentence 컬럼 선택
filtered_sentences = []

for index, row in train.iterrows():
    sentence_label = row['label']
    sentence_column = f'sentence{sentence_label}'
    filtered_sentences.append(row[sentence_column])

train_human = pd.DataFrame({
    'id': train['id'],
    'label': train['label'],
    'sentence_human': filtered_sentences
})

train_human
'''

"\n# train dataframe을 분석하기 좋게 변환하기\n# 레이블에 맞는 sentence 컬럼 선택\nfiltered_sentences = []\n\nfor index, row in train.iterrows():\n    sentence_label = row['label']\n    sentence_column = f'sentence{sentence_label}'\n    filtered_sentences.append(row[sentence_column])\n\ntrain_human = pd.DataFrame({\n    'id': train['id'],\n    'label': train['label'],\n    'sentence_human': filtered_sentences\n})\n\ntrain_human\n"

In [22]:
'''
# label에 해당하지 않는 sentence 컬럼들을 묶어서 train_ai 데이터프레임 생성
filtered_sentences_ai = []
for index, row in train.iterrows():
    label = row['label']
    sentences_ai = []
    for i in range(1, 5):
        if i != label:
            if f'sentence{i}' in row:
                sentences_ai.append(row[f'sentence{i}'])
            else:
                sentences_ai.append('')  # 해당하는 sentence가 없으면 빈 문자열 추가
    filtered_sentences_ai.append(sentences_ai)

# train_ai 데이터프레임 생성
train_ai = pd.DataFrame({
    'id': train['id'],
    'sentence1_ai': [sentences[0] for sentences in filtered_sentences_ai],
    'sentence2_ai': [sentences[1] if len(sentences) > 1 else '' for sentences in filtered_sentences_ai],
    'sentence3_ai': [sentences[2] if len(sentences) > 2 else '' for sentences in filtered_sentences_ai],
})

train_ai
'''

"\n# label에 해당하지 않는 sentence 컬럼들을 묶어서 train_ai 데이터프레임 생성\nfiltered_sentences_ai = []\nfor index, row in train.iterrows():\n    label = row['label']\n    sentences_ai = []\n    for i in range(1, 5):\n        if i != label:\n            if f'sentence{i}' in row:\n                sentences_ai.append(row[f'sentence{i}'])\n            else:\n                sentences_ai.append('')  # 해당하는 sentence가 없으면 빈 문자열 추가\n    filtered_sentences_ai.append(sentences_ai)\n\n# train_ai 데이터프레임 생성\ntrain_ai = pd.DataFrame({\n    'id': train['id'],\n    'sentence1_ai': [sentences[0] for sentences in filtered_sentences_ai],\n    'sentence2_ai': [sentences[1] if len(sentences) > 1 else '' for sentences in filtered_sentences_ai],\n    'sentence3_ai': [sentences[2] if len(sentences) > 2 else '' for sentences in filtered_sentences_ai],\n})\n\ntrain_ai\n"

In [23]:
'''
# 문장 길이 계산 및 분포 시각화
train_human['sentence_length'] = train_human['sentence_human'].apply(lambda x: len(x.split()))
sentence_lengths = train_human['sentence_length']

# 문장 길이 분포 시각화
plt.hist(sentence_lengths, bins=10, edgecolor='black')
plt.title("Distribution of Sentence Length of Human's")
plt.xlabel('Sentence Length')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()
'''

'\n# 문장 길이 계산 및 분포 시각화\ntrain_human[\'sentence_length\'] = train_human[\'sentence_human\'].apply(lambda x: len(x.split()))\nsentence_lengths = train_human[\'sentence_length\']\n\n# 문장 길이 분포 시각화\nplt.hist(sentence_lengths, bins=10, edgecolor=\'black\')\nplt.title("Distribution of Sentence Length of Human\'s")\nplt.xlabel(\'Sentence Length\')\nplt.ylabel(\'Frequency\')\nplt.grid(True)\nplt.show()\n'

In [24]:
'''
# 각 열의 문장 길이 계산
train_ai['sentence_length'] = filtered_sentences_ai.apply(lambda x: len(str(x).split()))
sentence_lengths = train_ai['sentence_length']

# 문장 길이 분포 시각화
plt.hist(sentence_lengths, bins=10, edgecolor='black')
plt.title("Distribution of Sentence Length of AI's")
plt.xlabel('Sentence Length')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()
'''

'\n# 각 열의 문장 길이 계산\ntrain_ai[\'sentence_length\'] = filtered_sentences_ai.apply(lambda x: len(str(x).split()))\nsentence_lengths = train_ai[\'sentence_length\']\n\n# 문장 길이 분포 시각화\nplt.hist(sentence_lengths, bins=10, edgecolor=\'black\')\nplt.title("Distribution of Sentence Length of AI\'s")\nplt.xlabel(\'Sentence Length\')\nplt.ylabel(\'Frequency\')\nplt.grid(True)\nplt.show()\n'

In [25]:
'''
nltk.download('punkt')

train['num_words']=train['text'].apply(lambda x:len(get_words(x)))
train['num_;']=train['text'].apply(lambda x: count_tokens(x, ';'))
train['num_"']=train['text'].apply(lambda x: count_tokens(x, '”'))
train['num_.']=train['text'].apply(lambda x: count_tokens(x, '.'))
train['num_,']=train['text'].apply(lambda x: count_tokens(x, ','))
train['num_punctuations'] =train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

test['num_words']=test['text'].apply(lambda x:len(get_words(x)))
test['num_;']=test['text'].apply(lambda x: count_tokens(x, ';'))
test['num_"']=test['text'].apply(lambda x: count_tokens(x, '"'))
test['num_.']=test['text'].apply(lambda x: count_tokens(x, '.'))
test['num_,']=test['text'].apply(lambda x: count_tokens(x, ','))
test['num_punctuations'] =test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
'''

'\nnltk.download(\'punkt\')\n\ntrain[\'num_words\']=train[\'text\'].apply(lambda x:len(get_words(x)))\ntrain[\'num_;\']=train[\'text\'].apply(lambda x: count_tokens(x, \';\'))\ntrain[\'num_"\']=train[\'text\'].apply(lambda x: count_tokens(x, \'”\'))\ntrain[\'num_.\']=train[\'text\'].apply(lambda x: count_tokens(x, \'.\'))\ntrain[\'num_,\']=train[\'text\'].apply(lambda x: count_tokens(x, \',\'))\ntrain[\'num_punctuations\'] =train[\'text\'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))\n\ntest[\'num_words\']=test[\'text\'].apply(lambda x:len(get_words(x)))\ntest[\'num_;\']=test[\'text\'].apply(lambda x: count_tokens(x, \';\'))\ntest[\'num_"\']=test[\'text\'].apply(lambda x: count_tokens(x, \'"\'))\ntest[\'num_.\']=test[\'text\'].apply(lambda x: count_tokens(x, \'.\'))\ntest[\'num_,\']=test[\'text\'].apply(lambda x: count_tokens(x, \',\'))\ntest[\'num_punctuations\'] =test[\'text\'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))\n'

## 모델링 KoGPT

In [26]:
!pip install transformers

In [27]:
!pip install torch

In [28]:
import torch

In [29]:
torch.cuda.is_available()

True

In [30]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    train['text'].tolist(),
    train['label'].tolist(),
    test_size=0.2
)
X_test = test['text'].tolist()

In [31]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [32]:
# KoGPT 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

# 패딩 토큰 추가
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# 토큰화 함수 정의
def tokenize_function(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

In [33]:
# 토큰화된 입력 준비
train_encodings = tokenize_function(X_train)
val_encodings = tokenize_function(X_val)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [34]:
# 데이터셋 클래스 정의
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        if self.labels:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

In [35]:
# 데이터셋 객체 생성
train_dataset = TextDataset(train_encodings, y_train)
val_dataset = TextDataset(val_encodings, y_val)

In [36]:
# KoGPT 모델 로드
model = AutoModelForSequenceClassification.from_pretrained('skt/kogpt2-base-v2', num_labels=2)
model.resize_token_embeddings(len(tokenizer))

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(51202, 768)

In [37]:
!pip install accelerate -U

In [38]:
!pip install transformers[torch] -U

In [39]:
# 훈련 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [40]:
# 모델 훈련
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.468800,0.451234
2,0.298200,0.356202
3,0.255700,0.312099


TrainOutput(global_step=60, training_loss=0.4183270295461019, metrics={'train_runtime': 9.2468, 'train_samples_per_second': 51.91, 'train_steps_per_second': 6.489, 'total_flos': 30865678663680.0, 'train_loss': 0.4183270295461019, 'epoch': 3.0})

In [44]:
# test 데이터셋 예측 수행 함수
def predict_function(texts):
    encodings = tokenize_function(texts)
    dataset = TextDataset(encodings)
    predictions = trainer.predict(dataset)
    return predictions.predictions

In [45]:
# index 별로 그룹화
grouped = test.groupby('index')

# 결과를 저장할 리스트
results = []

# 각 그룹에 대해 예측 수행
for idx, group in grouped:
    texts = group['text'].tolist()
    sentence_numbers = group['sentence'].str.extract(r'(\d+)').astype(int).values.flatten()

    # 예측 수행
    logits = predict_function(texts)
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=1)[:, 1].numpy()  # 클래스 1에 대한 확률값

    # 상위 두 문장의 인덱스
    top_two_indices = probs.argsort()[-2:][::-1]
    top_two_sentences = sentence_numbers[top_two_indices]

    # id와 label 생성
    id_val = f"TEST_{str(idx).zfill(4)}"
    label_val = ''.join(map(str, top_two_sentences))

    results.append({'id': id_val, 'label': label_val})

# 결과 데이터프레임 생성
submission = pd.DataFrame(results)

In [46]:
submission

,id,label
0,TEST_0000,42
1,TEST_0001,13
2,TEST_0002,24
3,TEST_0003,13
4,TEST_0004,32
...,...,...
1095,TEST_1095,21
1096,TEST_1096,24
1097,TEST_1097,41
1098,TEST_1098,43


In [48]:
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/dataset/text_classification/submission_kogpt.csv', index=False)

점수: 0.918

## 모델링 EiLLM
: 한국어모델(polyglot-ko-12.8b)과 영어 모델(llmama2-13b)을 기반으로 약 63만개의 instruction following 데이터로 fine-tuning된 모델
https://aiopen.etri.re.kr/eiLLModel

In [49]:
import torch
from transformers import pipeline, AutoModelForCausalLM

In [50]:
MODEL = 'etri-xainlp/polyglot-ko-12.8b-instruct'

In [51]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
).to(device=f"cuda", non_blocking=True)
model.eval()

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/26.0G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30003, 5120)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-39): 40 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXSdpaAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=5120, out_features=15360, bias=True)
          (dense): Linear(in_features=5120, out_features=5120, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=5120, out_features=20480, bias=True)
          (dense_4h_to_h): Linear(in_features=20480, out_features=5120, bias=T

In [52]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=MODEL,
    device=0
)
pipe.model.config.pad_token_id = pipe.model.config.eos_token_id

tokenizer_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
